<a href="https://colab.research.google.com/github/dawads/bibliometri/blob/master/network_data_from_diva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Vi använder Pandas och Numpy för beräkning och databearbetning i Python
import pandas as pd
import numpy as np

Import av poster via url-sök i DiVA, csvall2

In [0]:
url=(
  'http://liu.diva-portal.org/smash/export.jsf?'+
  'format=csvall2&addFilename=true&aq=[[]]'+
  '&aqe=[]'+
  '&aq2=[[{"organisationId":"276","organisationId-Xtra":true},{"publicationTypeCode":["review","article"]},{"publicationStatus":"published"},{"contentTypeCode":["refereed"]}]]'+
  '&onlyFullText=false'+
  '&noOfRows=500'+
  '&sortOrder=title_sort_asc'+
  '&sortOrder2=title_sort_asc'
)
divaData=pd.read_csv(url)


Ifrån CSV-filen skapar vi en dataframe som har en rad per PID och ämnesord

In [15]:
headers=list(divaData.columns.values)
#print(headers)

divaDataTwo=divaData[['PID','Categories']]
divaDataTwo['Categories']=divaData['Categories'].str.split(';',-1,False)
#print(divaDataTwo)

subjects=divaDataTwo.set_index(['PID'])['Categories'].apply(pd.Series).stack()

subjects=subjects.reset_index()
#print(subjects)
subjects.columns=['PID','Order','Categories']
subjects.head()

#test=subjects.unique()
#print(allSubjects)
#test=allSubjects.unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,PID,Order,Categories
0,266537,0,Teknik och teknologier (2)
1,242842,0,Teknik och teknologier (2)
2,634428,0,Acceleratorfysik och instrumentering (10306)
3,634430,0,Acceleratorfysik och instrumentering (10306)
4,1236429,0,Annan materialteknik (20599)


Nu skapar vi en nodes-tabell som kan exporteras till Gephi med radnummer, namn och vikt

In [13]:
nodes = subjects.groupby('Categories').agg({'Categories': np.size})
nodes['Weight'] = nodes['Categories']
nodes['Size'] = nodes['Categories'].apply(np.sqrt)
nodes = nodes[['Weight', 'Size']].reset_index()
nodes['order'] = nodes.index
nodes = nodes[['order','Categories', 'Weight','Size']]
nodes.head()
#having count > X ?

,order,Categories,Weight,Size
0,0,Acceleratorfysik och instrumentering (10306),3,1.732051
1,1,Algebra och logik (10103),1,1.000000
2,2,Allmänmedicin (30224),3,1.732051
3,3,Analytisk kemi (10401),2,1.414214
4,4,Annan data- och informationsvetenskap (10299),1,1.000000


nodes AS ( SELECT row_number() OVER (ORDER BY Categories) AS id, Categories AS label, count() AS p FROM subjects GROUP BY Categories HAVING count() > 0 ),

id label p sqrt_p
1 Barcelona Hospital Campus Vall d’Hebron, Barcelona 3619 60.1581249707801
2;Charité-Universitätsmedizin;14572;120.714539306581
3;Erasmus MC University Medical Centre;14888;122.016392341357

edges AS
(
 SELECT n1.id AS source, n2.id AS target, count(*) AS weight
 FROM nodes n1
 INNER JOIN pek p1 ON (p1.org = n1.label)
 INNER JOIN pek p2 ON (p2.id = p1.id)
 INNER JOIN nodes n2 ON (n2.label = p2.org AND n2.id > n1.id)
 GROUP BY n1.id, n2.id
)

In [33]:
# df1.merge(df2.merge(df3, on='date'), on='date')
# df1.merge(df2, on='date')
edges_l = nodes.merge(subjects, how='inner', on='Categories')
edges_i = subjects.merge(subjects, how='inner', on='PID')
edges_li = edges_l.merge(edges_i, how='inner', on='PID')
edges = edges_li.merge(nodes, how='inner', on='Categories')
#edges = edges[edges['order'] > edges['order']]

edges.head(15)
#edges_l.head()

#df = pd.merge(A, B, how='inner', left_on='cusip', right_on='ncusip')
#df = df[(df['fdate']>=df['namedt']) & (df['fdate']<=df['nameenddt'])]

,order_x,Categories,Weight_x,Size_x,PID,Order,Order_x,Categories_x,Order_y,Categories_y,order_y,Weight_y,Size_y
0,0,Acceleratorfysik och instrumentering (10306),3,1.732051,634428,0,0,Acceleratorfysik och instrumentering (10306),0,Acceleratorfysik och instrumentering (10306),0,3,1.732051
1,0,Acceleratorfysik och instrumentering (10306),3,1.732051,634430,0,0,Acceleratorfysik och instrumentering (10306),0,Acceleratorfysik och instrumentering (10306),0,3,1.732051
2,0,Acceleratorfysik och instrumentering (10306),3,1.732051,1219969,0,0,Acceleratorfysik och instrumentering (10306),0,Acceleratorfysik och instrumentering (10306),0,3,1.732051
3,1,Algebra och logik (10103),1,1.000000,1188675,0,0,Algebra och logik (10103),0,Algebra och logik (10103),1,1,1.000000
4,2,Allmänmedicin (30224),3,1.732051,1048661,0,0,Allmänmedicin (30224),0,Allmänmedicin (30224),2,3,1.732051
5,2,Allmänmedicin (30224),3,1.732051,615892,2,0,Pediatrik (30221),0,Pediatrik (30221),2,3,1.732051
6,2,Allmänmedicin (30224),3,1.732051,615892,2,0,Pediatrik (30221),1,Dermatologi och venereologi (30204),2,3,1.732051
7,2,Allmänmedicin (30224),3,1.732051,615892,2,0,Pediatrik (30221),2,Allmänmedicin (30224),2,3,1.732051
8,2,Allmänmedicin (30224),3,1.732051,615892,2,1,Dermatologi och venereologi (30204),0,Pediatrik (30221),2,3,1.732051
9,2,Allmänmedicin (30224),3,1.732051,615892,2,1,Dermatologi och venereologi (30204),1,Dermatologi och venereologi (30204),2,3,1.732051


In [0]:
(nodes.assign(row_id=subjects.sort_values(['Categories'], ascending=False)
              .rank(method='first', ascending=False))
              #.cumcount() + 1)
)

ValueError: ignored

https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html